In [25]:
import os
import torch
import torch.nn as nn
from torchvision import transforms

In [26]:
import sys
sys.path.insert(1, "../../../")

In [27]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.yolov8cls_path import Model

In [28]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [29]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [30]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [31]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [32]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

# Inference

In [33]:
model = Model(num_classes=10, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [34]:
img = train_dataset[0][0].unsqueeze(0)

In [35]:
from torch.profiler import profile, ProfilerActivity

In [36]:
prof = profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True)

prof.start()
for i in range(12):
    out = model(img)
    prof.step()
prof.stop()
print(prof.key_averages().table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total KFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   ProfilerStep*         5.25%      15.476ms       100.00%     294.617ms      29.462ms       6.43 Mb    -268.71 Mb            10            --  
                      Conv block        14.82%      43.674ms        90.15%     265.590ms       1.022ms     243.03 Mb           0 b           260            --  
                    aten::conv2d         0.71%       2.086ms        55.56%     163.694ms     629.590us      80.87 Mb           0 b           260  15245475.840  
               aten::convolution  